<a href="https://colab.research.google.com/github/anythingispossible19/SP19/blob/master/Enigma.py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import itertools, setting data generation (Part 2)
from itertools import product
from itertools import permutations
from itertools import combinations

# Part 1, Enigma Simulation

# Helper Component 
class HelperComponent:
   
    def CharacterToOrdinal(self, character): 
    
        # Map A-Z to 0-25 via ASCII function ord - 65 (shift down from A to 0)   
        return (ord(character) - 65) 
            
    def OrdinalToCharacter(self, character): 
    
        # Map 0-25 to A-Z via ASCII functions chr + 65 (shift up from 0 to A)
        return chr(character + 65)       
    
    # Constuctor 
    def __init__(self):                                    

        # Rotor data as identifier, right - left wiring (from which the reverse can be calculated via index function of lists), notch 
        self.Rotors = { "I":    ("EKMFLGDQVZNTOWYHXUSPAIBRCJ", "Q"), 
                        "II":   ("AJDKSIRUXBLHWTMCQGZNPYFVOE", "E"), 
                        "III":  ("BDFHJLCPRTXVZNYEIWGAKMUSQO", "V"),
                        "IV":   ("ESOVPZJAYQUIRHXLNFTGKDCMWB", "J"), 
                        "V":    ("VZBRGITYUPSDNHLXAWMJQOFECK", "Z"), 
                        "Beta": ("LEYJVCNIXWPBQMDRTAKZGFUHOS",  ""),  
                        "Gamma":("FSOKANUERHMBTIYCWLQPZXVGJD",  ""),  
                        "A":    ("EJMZALYXVBWFCRQUONTSPIKHGD",  ""),
                        "B":    ("YRUHQSLDPXNGOKMIEBFZCWVJAT",  ""), 
                        "C":    ("FVPJIAOYEDRZXWGCTKUQSBNMHL",  "")}  
        
# Pluglead    
class PlugLead(HelperComponent): 

    # Constuctor for pluglead
    def __init__(self, pair):     
    
        # Map character pair to - 0 25 via base helper function 
        first, second = map(self.CharacterToOrdinal, pair)  
        
        # Swap 
        self.firstInPair = second
        self.secondInPair = first         
                      
    def encode(self, character):          
    
        # Map character to - 0 25 
        mappedCharacter = self.CharacterToOrdinal(character)
                
        # Swap if character is firstInPair
        if mappedCharacter == self.firstInPair: 
            return self.OrdinalToCharacter(self.secondInPair)  
            
        # Swap if character is secondInPair
        if mappedCharacter == self.secondInPair: 
            return self.OrdinalToCharacter(self.firstInPair)
        
        # Return character if no map 
        return character 

# Plugboard        
class Plugboard(HelperComponent): 

    # Constuctor for plugboard 
    def __init__(self):  
    
        # initialise empty list for plugleads 
        self.pluglead = []
    
    # Add a pluglead 
    def add(self, pluglead):
    
        # Add pluglead object to list 
        self.pluglead.append(pluglead)
            
    # Encode a character via iterate the plugleads     
    def encode(self, character):
    
        # Check the plugleads for swaps required 
        for pair in self.pluglead:      
        
            # Encode
            mappedCharacter = pair.encode(character)
            
            # If different then we have a swap so exit else continue
            if mappedCharacter != character: 
                return mappedCharacter
                
        # Return if no match
        return character        

# Rotor 
class Rotor(HelperComponent):

    # Construct a rotor 
    def __init__(self, rotorIdentifier, offset = "A", ringSetting = "01", debug = True): 
    
        # Get the rotor list 
        super().__init__()
        
        # Rotor set up wiring via data in base class list 
        self.rotor = rotorIdentifier
        (rotorIdentifier, notch) =  self.Rotors[rotorIdentifier]      
        self.forwardWiring = list(map(self.CharacterToOrdinal, rotorIdentifier))       
        self.reverseWiring = [self.forwardWiring.index(i) for i in range(26)] 
        
        # Ring validation: "Increasing the ring setting has the exact same effect as decreasing the position setting. 
        # It shifts the internal wiring in the opposite direction."
        ringSetting = int(ringSetting)
       
        # Notch validation 
        self.notch = ""        
        if notch:              
            self.notch = ((self.CharacterToOrdinal(notch) - ringSetting + 1) % 26) 
        
        # Offset management 
        self.offset = self.CharacterToOrdinal(offset) - ringSetting + 1  
        
        # Ring setting management 
        self.ringSetting = ringSetting - 1
        
        # Debug option 
        if debug == True:         
#             print("rotor:", self.rotor)
#             print("forwardWiring:", self.forwardWiring)
#             print("reverseWiring:", self.reverseWiring)
#             if self.notch:
#                 print("notch:", self.notch)
#             print("offset:", self.offset)
#             print("ringSetting:", self.ringSetting)
    
    # Notch 
    def notchCheck(self):     
        return (self.offset % 26) == self.notch
    
    # Advance 
    def advance(self):     
        self.offset += 1
        
    # Right left: test function 
    def RightLeft(self, character, offset = 0):     
        mapping = self.CharacterToOrdinal(character)
        return self.OrdinalToCharacter(self.forwardWiring[(mapping + self.offset - offset) % 26])
    
    # Left right: test function   
    def LeftRight(self, character, offset = 0):     
        mapping = self.CharacterToOrdinal(character)
        return self.OrdinalToCharacter(self.reverseWiring[(mapping + self.offset - offset) % 26]) 
    
    # Enter: Right left, ordinal mapping 
    def Enter(self, character, offset = 0):     
        return self.forwardWiring[(character + self.offset - offset) % 26]
    
    # Exit: Left right, ordinal mapping 
    def Exit(self, character, offset = 0):     
        return self.reverseWiring[(character + self.offset - offset) % 26]   
  
# Enigma machine hosting all components  
class EnigmaMachine(HelperComponent):

    # Construct enigma machine 
    def __init__(self, plugboardString, *rotors): 

        # Plugboard 
        self.plugboard = Plugboard()
        
        if plugboardString: 
            # Populate plugboard with plugleads
            for pair in plugboardString.split(" "):
                self.plugboard.add(PlugLead(pair))
        
        # Rotors, including reflector 
        self.rotors = rotors 
        
    def encode(self, message):
        
        # Result holder
        output = ""
        
        # Iterate string 
        for character in message:

            # get the rotors that rotate only
            left = self.rotors[-3]
            middle = self.rotors[-2]
            right = self.rotors[-1]
            
            # rotation management 
            if middle.notchCheck():
                # move two 
                left.advance()
                middle.advance()
            elif right.notchCheck():
                # move one 
                middle.advance()
            # always move 
            right.advance()
            
            # offset tracker, local 
            offset = 0
                        
            # Plugboard after conversion to ordinal from character  
            character = self.CharacterToOrdinal(self.plugboard.encode(character))
                       
            # right to left flow of character 
            for rotor in self.rotors[::-1]: 
                character = rotor.Enter(character, offset)
                # get new offset
                offset = rotor.offset
                
            #left to right flow of character 
            for rotor in self.rotors[1:]:      
                character = rotor.Exit(character, offset)
                # get new offset 
                offset = rotor.offset
                
            # offset management 
            character = (character - offset) % 26 
            
            # Plugboard after conversion to character from ordinal 
            character = self.plugboard.encode(self.OrdinalToCharacter(character)) 
                        
            # Result concatenation
            output += character
                    
        return output

# Part 2, Code Breaking         

# Part 2, Code 1 break management class       
"""
Code 1
You recovered an Enigma machine! Amazingly, it is set up in that day's position, ready for you to replicate in your software. But unfortunately the label has worn off the reflector. All the other settings are still in place, however. You also found a book with the title "SECRETS" which contained the following code, could it be so simple that the code contains that text?

* Code: `DMEXBMKYCVPNQBEDHXVPZGKMTFFBJRPJTLHLCHOTKOYXGGHZ`
* Crib: `SECRETS`

* Rotors: `Beta Gamma V`
* Reflector: Unknown
* Ring settings: `04 02 14`
* Starting positions: `MJM`
* Plugboard pairs: `KI XN FL`  
"""
class Code1Breaker(HelperComponent):
        
    # Pass in all required data 
    def __init__(self, text, reflectorIds, crib):
    
        # for each reflector 
        for r in reflectorIds:
        
            # turn off debug statements 
            reflector = Rotor(r,    "A", "01", False)            
            Beta =  Rotor("Beta",   "M", "04", False)
            Gamma = Rotor("Gamma",  "J", "02", False)
            V =     Rotor("V",      "M", "14", False)

            # decode attempt 
            machine = EnigmaMachine("KI XN FL", reflector, Beta, Gamma, V)
            result = machine.encode(text) 

            # crib found in decoded message?
            if result.find(crib) > -1: 
                self.result = result 
                self.reflector = r
    
# Part 2, Code 2 break management class    
"""
Code 2
You leave the machine in the hands of the university. The team have cracked the day's settings thanks to some earlier codebreaking, but unfortunately, the initial rotor positions are changed for each message. For the message below, the team has no idea what the initial settings should be, but know the message was addressed to them. Help them out.

* Code: `CMFSUPKNCBMUYEQVVDYKLRQZTPUFHSWWAKTUGXMPAMYAFITXIJKMH`
* Crib: `UNIVERSITY`

* Rotors: `Beta I III`
* Reflector: `B`
* Ring settings: `23 02 10`
* Starting positions: Unknown
* Plugboard pairs: `VH PT ZG BJ EY FS`  
""" 
class Code2Breaker(HelperComponent):    
    
    # Pass in all required data     
    def __init__(self, text, crib):             
      
        # Get all combinations of A-Z, 3 start positions 
        #https://docs.python.org/3/library/itertools.html#itertools.product
        self.crossproductOfAlphabet = list(product('ABCDEFGHIJKLMNOPQRSTUVWXYZ', repeat = 3))
        self.count = len(self.crossproductOfAlphabet)

        # for each combination 
        for combination in self.crossproductOfAlphabet: 
       
            # turn off debug statements 
            reflector = Rotor("B", "A", "01", False)            
            Beta =  Rotor("Beta",combination[0], "23", False)
            I =     Rotor("I",   combination[1], "02", False)
            III =   Rotor("III", combination[2], "10", False)

            # decode attempt 
            machine = EnigmaMachine("VH PT ZG BJ EY FS", reflector, Beta, I, III)
            result = machine.encode(text) 

            # crib found in decoded message?
            if result.find(crib) > -1: 
                self.result = result 
                self.startingPositions = combination
    
# Part 2, Code 3 break management class      
"""
Code 3
The department has intercepted a message from the admissions team. They know it contains the word "THOUSANDS" and they are worried it might relate to how many students are arriving next semester. But the admissions team are a bit unusual: they love even numbers, and hate odd numbers. You happen to know they will never use an odd-numbered rotor, ruling out I, III, and V. They will also never use a ring setting that has even a single odd digit: 02 is allowed but 11 is certainly not, and even 12 is banned.

Code: ABSKJAKKMRITTNYURBJFWQGRSGNNYJSDRYLAPQWIAGKJYEPCTAGDCTHLCDRZRFZHKNRSDLNPFPEBVESHPY
Crib: THOUSANDS
Rotors: Unknown but restricted (see above)
Reflector: Unknown
Ring settings: Unknown but restricted (see above)
Starting positions: EMY
Plugboard pairs: FH TS BE UQ KD AL 
"""
class Code3Breaker(HelperComponent):

    # Ensure rotors are unique e.g. False: ('A', 'II', 'II', 'II', '02', '02', '02')
    def duplicateCheck(self, combination):
       for element in combination:
          if combination.count(element) > 1:
             return True
          return False
            
    # Pass in all required data   
    def __init__(self, text, crib, debug = False):
    
        # reflector 
        self.reflector =    ["A", "B", "C"]
        
        # rotor 
        self.rotorLeft =    ["II", "IV", "Beta", "Gamma"]
        self.rotorMiddle =  ["II", "IV", "Beta", "Gamma"]
        self.rotorRight =   ["II", "IV", "Beta", "Gamma"]       
        
        # ring 
        self.ringLeft =     ["02", "04", "06", "08", "20", "22", "24", "26"]
        self.ringMiddle =   ["02", "04", "06", "08", "20", "22", "24", "26"]
        self.ringRight =    ["02", "04", "06", "08", "20", "22", "24", "26"]
    
        # Get all combinations of rotor, reflector and ring settings  
        #https://docs.python.org/3/library/itertools.html#itertools.product
        self.crossproductOfSets = list(product(self.reflector, 
                                                self.rotorLeft, 
                                                self.rotorMiddle, 
                                                self.rotorRight, 
                                                self.ringLeft, 
                                                self.ringMiddle, 
                                                self.ringRight))
        # Count parameter tuples 
        self.count = len(self.crossproductOfSets) 

        # counter 
        countNotUnique = 0
        
        # results 
        self.result = "" 
        self.combinationOfReflectorRotorRing = ""

        # for each combination 
        for combination in self.crossproductOfSets:            
                      
            # check all rotor elements unique else continue 
            if self.duplicateCheck(combination[1:4]): 
                if debug == True:
                    print("Parameter combination:", combination)
                    print("Rotor combination:", combination[1:4]) 
                    print("Rotors are not unique, so skip this combination.")
                    countNotUnique += 1
                    continue
                else:
                    countNotUnique += 1
                    continue                    

            # Get reflector & rotors 
            reflector = Rotor(combination[0], "A", "01", False)            
            left    =   Rotor(combination[1], "E", combination[4], False)
            middle  =   Rotor(combination[2], "M", combination[5], False)
            right   =   Rotor(combination[3], "Y", combination[6], False)

            # decode attempt 
            machine = EnigmaMachine("FH TS BE UQ KD AL", reflector, left, middle, right)
            result = machine.encode(text)            
                
            # crib found in decoded message?
            if result.find(crib) > -1: 
                self.result = result 
                self.combinationOfReflectorRotorRing = combination
            
        # Duplicated rotor combinations 
        self.countNotUnique = countNotUnique
            
# Part 2, Code 4 break management class     
"""
Code 4
On my way home from working late as I walked past the computer science lab I saw one of the tutors playing with the Enigma machine. Mere tutors are not allowed to touch such important equipment! Suspicious, I open the door, but the tutor hears me, and jumps out of the nearest window. They left behind a coded message, but some leads have been pulled out of the machine. It might contain a clue, but I'll have to find the missing lead positions (marked with question marks in the settings below).

Code: SDNTVTPHRBNWTLMZTQKZGADDQYPFNHBPNHCQGBGMZPZLUAVGDQVYRBFYYEIXQWVTHXGNW
Crib: TUTOR
Rotors: V III IV
Reflector: A
Ring settings: 24 12 10
Starting positions: SWU
Plugboard pairs: WP RJ A? VF I? HN CG BS
"""  
class Code4Breaker(HelperComponent):

    def convertToString(self, sequence, seperator):
       result = seperator.join(sequence)
       return result

    # Pass in all required data     
    def __init__(self, text, crib, debug = False):             
      
        # Get all combinations of pairs in the alphabet  
        #https://docs.python.org/3/library/itertools.html#itertools.product
        self.crossproductOfAlphabet = list(product('ABCDEFGHIJKLMNOPQRSTUVWXYZ', repeat = 2))
        self.count = len(self.crossproductOfAlphabet)
        
        # result list 
        self.resultList = []

        # for each combination 
        for combination in self.crossproductOfAlphabet:     
            
            # plugboard 
            plugboard = "WP RJ A? VF I? HN CG BS"
            if debug == True: 
#                 print(plugboard) 
                                                   
            # turn off debug statements 
            reflector = Rotor("A",     "A", "01", False)            
            V =         Rotor("V",     "S", "24", False)
            III =       Rotor("III",   "W", "12", False)
            IV =        Rotor("IV",    "U", "10", False)
            
            # replace '?' characters in list 
            plugboardPopulated = list(plugboard)
            plugboardPopulated[7] = combination[0]
            plugboardPopulated[13] = combination[1]  
            plugboardString = self.convertToString(plugboardPopulated, "")            
            if debug == True: 
#                 print(plugboardString) 
            
            # decode attempt 
            machine = EnigmaMachine(plugboardString, reflector, V, III, IV)
            result = machine.encode(text) 
            
            # crib found in decoded message?
            if result.find(crib) > -1: 
                self.resultList.append(result + ", (" + combination[0] + ", " + combination[1] + ")")
                
        # Show the set for inspection 
        for element in self.resultList: 
#             print(element)
            
# Part 2, Code 5 break management class   
"""
Code 5
I later remembered that I had given the tutor permission to use the Enigma machine to solve some codes I'd received via email. As for the window, they are just a big fan of parkour, this is always how they leave the building. It seems they are stuck on one last code. It came in via email so we suspect it's just spam, probably related to a social media website, but you never know when you'll find a gem in that kind of stuff.

The tutor has narrowed the search and found most of the settings, but it seems this code was made with a non-standard reflector. Indeed, there was a photo attached to the email along with the code. It appears that the sender has taken a standard reflector, cracked it open, and swapped some of the wires – two pairs of wires have been modified, by the looks of the dodgy soldering job.

To be clear, a single wire connects two letters, e.g. mapping A to Y and Y to A. The sender has taken two wires (fours pairs of letters), e.g. A-Y and H-J, and swapped one of the ends, so one option would be H-Y and A-J. They did this twice, so they modified eight letters total (they did not swap the same wire more than once).

In your answer, include what the original reflector was and the modifications.

Code: HWREISXLGTTBYVXRCWWJAKZDTVZWKBDJPVQYNEQIOTIFX
Crib: the name of a social media website/platform
Rotors: V II IV
Reflector: Unknown and non-standard (see above)
Ring settings: 06 18 07
Starting positions: AJL
Plugboard pairs: UG IE PO NX WT
"""
class Code5Breaker(HelperComponent):

    # brute force swaps 
    def bruteForceSwaps(self, text, crib):
    
        # result holder 
        self.resultList = []
        
        # 1. get reflector list 
        reflectorIds = ['A', 'B', 'C']  
        
        # 2. for each reflector 
        for r in reflectorIds:  
        
            # get reflector; default options specified 
            reflector = Rotor(r, "A", "01", False)
            
            # rotor set 
            V =  Rotor("V",  "A", "06", False)
            II = Rotor("II", "J", "18", False)
            IV = Rotor("IV", "L", "07", False) 
            
            # pairs holder 
            pairedIndices = []   
            
            # from reflector wiring, get list of pairs, length 26 / 2, ensuring no repeats in pairs e.g. (0, 4) and (4, 0) tuples 
            for character in reflector.forwardWiring:
                a = reflector.forwardWiring.index(character)
                b = character 
                reverseA = b 
                reverseB = a 
                if not (reverseA, reverseB) in pairedIndices:
                    pairedIndices.append((a, b))
           
            # get index (0-12) combinations to represent all possible 2 pair (4 element) selections from this list of 13 
            # https://docs.python.org/3/library/itertools.html#itertools
            self.combinationsOfReflectorPairs = list(combinations(range(13), r = 4)) 
            self.count = len(self.combinationsOfReflectorPairs)
                        
            # we now have a representation of 715 selections of 2 unique pairs as a list of (a, b, c, d) 
            for pairSelection in self.combinationsOfReflectorPairs:            
                
                # we now have the reflector as pairedIndices and the pairs as a list of indices, as pairSelection                 
                # we now want to create combinations of the pairs to simulate wire swapping               
                
                # group 1 
                group1Pair1Index = pairSelection[0]
                group1Pair2Index = pairSelection[1] 
                group1Pair1 = pairedIndices[group1Pair1Index]
                group1Pair2 = pairedIndices[group1Pair2Index]
                
                result1 = ''
                result1 += self.OrdinalToCharacter(group1Pair1[0])
                result1 += self.OrdinalToCharacter(group1Pair1[1])
                result1 += self.OrdinalToCharacter(group1Pair2[0])
                result1 += self.OrdinalToCharacter(group1Pair2[1])
                
                # combine 1
                self.combinationsOfPairOne = list(combinations(result1, r = 2))                
                
                # group 2 
                group2Pair1Index = pairSelection[2]
                group2Pair2Index = pairSelection[3] 
                group2Pair1 = pairedIndices[group2Pair1Index]
                group2Pair2 = pairedIndices[group2Pair2Index]   
                
                result2 = ''
                result2 += self.OrdinalToCharacter(group2Pair1[0])
                result2 += self.OrdinalToCharacter(group2Pair1[1])
                result2 += self.OrdinalToCharacter(group2Pair2[0])
                result2 += self.OrdinalToCharacter(group2Pair2[1])
                
                # combine 2
                self.combinationsOfPairTwo = list(combinations(result2, r = 2))  
                
                # for each of 1, 2, 3, 4 swap configs of the reflector under analysis: 
                # iterate pair 1 (outer loop) and pair 2 (inner loop)
                # for one in self.combinationsOfPairOne:
                #    print(one)  
                #    for two in self.combinationsOfPairTwo: 
                #        print(two)
                # flatten reflector to process in correct format (flattenReflector)                       
                # get reflector object                         
                # assign flattened reflector to reflector object: reflector.forwardWiring and reflector.reverseWiring 
                # run EnigmaMachine                         
                # store results and config. if break code            
        
    # flatten data 
    def flattenReflector(self, reflector): 
    
        # holds original format of reflector  
        result = [None] * 26
        
        # for each pair, assign values to correct index 
        for pair in reflector:
            result[pair[0]] = pair[1]
            result[pair[1]] = pair[0]
        
        # return 
        return result    
        
    # Pass in all required data   
    def __init__(self, text, crib, debug = False):
    
        # run swapping algorithm 
        self.bruteForceSwaps(text, crib)         
            
# Main entry point    
if __name__ == "__main__":
    pass